In [1]:
!pip install pyspark==3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 14.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805893 sha256=d272d7b6920c5e9dddba364917315a15e7d0f01b8fba82f3bedc9149f96d3986
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
#Start spark sessnon and configureation
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[2]").appName("dat").getOrCreate()
sc = spark.sparkContext
#create an instance of SQLContext
sqlContext = SQLContext(spark)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 15:21:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.10/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# Import libraries and other functions
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import csv
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [4]:
LDNCrime = spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").\
csv("/kaggle/input/ldn-crime-dataset/LDN Crime Data")

In [5]:
LDNCrime.printSchema()

root
 |-- Crime ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported by: string (nullable = true)
 |-- Falls within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA code: string (nullable = true)
 |-- LSOA name: string (nullable = true)
 |-- Crime type: string (nullable = true)
 |-- Last outcome category: string (nullable = true)
 |-- Context: string (nullable = true)



In [6]:
# tidy up the column names

LDNCrime = LDNCrime.withColumnRenamed('Crime ID', 'Crime_ID')
LDNCrime = LDNCrime.withColumnRenamed('Reported by', 'Reported_by')
LDNCrime = LDNCrime.withColumnRenamed('Falls within', 'Falls_within')
LDNCrime = LDNCrime.withColumnRenamed('LSOA code', 'LSOA_code')
LDNCrime = LDNCrime.withColumnRenamed('LSOA name', 'LSOA_name')
LDNCrime = LDNCrime.withColumnRenamed('Crime type', 'Crime_type')
LDNCrime = LDNCrime.withColumnRenamed('Last outcome category', 'Last_outcome_category')

In [7]:
LDNCrime.printSchema()

root
 |-- Crime_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported_by: string (nullable = true)
 |-- Falls_within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA_code: string (nullable = true)
 |-- LSOA_name: string (nullable = true)
 |-- Crime_type: string (nullable = true)
 |-- Last_outcome_category: string (nullable = true)
 |-- Context: string (nullable = true)



In [8]:
LDNCrime.take(2)

[Row(Crime_ID='b45dac6a67d5db23714d7eb073a3482c63f45f27f4b6bd0c88117ff49753f1b2', Month='2021-06', Reported_by='Nottinghamshire Police', Falls_within='Nottinghamshire Police', Longitude=-1.400636, Latitude=53.075034, Location='On or near B6016', LSOA_code='E01019475', LSOA_name='Amber Valley 004E', Crime_type='Violence and sexual offences', Last_outcome_category='Unable to prosecute suspect', Context=None),
 Row(Crime_ID='63c3b35c31af90a96816edf97bb866d937b9cc2e03f99616daaa7d5dcd188be7', Month='2021-06', Reported_by='Nottinghamshire Police', Falls_within='Nottinghamshire Police', Longitude=-1.410348, Latitude=53.04264, Location='On or near Pear Tree Avenue', LSOA_code='E01019463', LSOA_name='Amber Valley 008F', Crime_type='Violence and sexual offences', Last_outcome_category='Unable to prosecute suspect', Context=None)]

In [9]:
#get the counts for each month ordered in descending order
LDNCrime.groupby("Month").count().orderBy(desc("count")).show()

+-------+-----+
|  Month|count|
+-------+-----+
|2021-06|32975|
|2021-07|32286|
|2021-05|30720|
|2021-03|30717|
|2021-08|30060|
|2021-04|29992|
|2021-09|29614|
|2021-11|29091|
|2021-10|28585|
|2021-12|27046|
|2021-01|26044|
|2021-02|25316|
+-------+-----+



In [10]:
#retrieve yearly amount of crimes sorted descendantly
LDNCrime.select(substring(LDNCrime.Month, 1,4).alias('Year'), "Month")\
.groupby("Year").count().show()

+----+------+
|Year| count|
+----+------+
|2021|352446|
+----+------+

